# Deep-Live-Cam GPU - Fixed Mouth Mask
GPU face swap with working mouth mask functionality

In [ ]:
# Setup
!apt update -qq && apt install -y ffmpeg
!git clone https://github.com/Mayank-kanojiya/liveswap.git
%cd liveswap
!pip install -q opencv-python onnxruntime-gpu torch torchvision insightface
!pip install -q customtkinter pillow psutil opennsfw2 protobuf cv2-enumerate-cameras
!pip install -q git+https://github.com/xinntao/BasicSR.git@master
!pip install -q git+https://github.com/TencentARC/GFPGAN.git@master

In [ ]:
# Download models
import os
os.makedirs('models', exist_ok=True)
!wget -q -O models/GFPGANv1.4.pth https://huggingface.co/hacksider/deep-live-cam/resolve/main/GFPGANv1.4.pth
!wget -q -O models/inswapper_128_fp16.onnx https://huggingface.co/hacksider/deep-live-cam/resolve/main/inswapper_128_fp16.onnx

In [ ]:
# Fixed face swap with working mouth mask
import cv2
import numpy as np
import sys
sys.path.append('.')

import modules.globals
modules.globals.headless = True
modules.globals.execution_providers = ['CUDAExecutionProvider', 'CPUExecutionProvider']
modules.globals.frame_processors = ['face_swapper']

from modules.processors.frame.face_swapper import get_face_swapper, process_frame
from modules.face_analyser import get_one_face, get_many_faces
from modules.utilities import is_video, extract_frames, create_video, restore_audio, get_temp_frame_paths, create_temp, clean_temp

face_swapper = get_face_swapper()

def create_mouth_mask(landmarks, image_shape):
    """Create mouth mask from facial landmarks"""
    try:
        # Mouth landmarks (68-point model)
        mouth_points = landmarks[48:68]  # Outer lip points
        
        # Create mask
        mask = np.zeros(image_shape[:2], dtype=np.uint8)
        cv2.fillPoly(mask, [mouth_points.astype(np.int32)], 255)
        
        # Dilate mask slightly
        kernel = np.ones((5,5), np.uint8)
        mask = cv2.dilate(mask, kernel, iterations=2)
        
        return mask
    except:
        return None

def face_swap_with_mouth_mask(source_path, target_path, output_path, mouth_mask=False, many_faces=False):
    modules.globals.mouth_mask = mouth_mask
    modules.globals.many_faces = many_faces
    modules.globals.source_path = source_path
    modules.globals.target_path = target_path
    modules.globals.output_path = output_path
    
    source_img = cv2.imread(source_path)
    source_face = get_one_face(source_img)
    
    if is_video(target_path):
        print("Processing video...")
        create_temp(target_path)
        extract_frames(target_path)
        
        temp_frame_paths = get_temp_frame_paths(target_path)
        
        for i, frame_path in enumerate(temp_frame_paths):
            frame = cv2.imread(frame_path)
            
            if mouth_mask:
                # Get target face for mouth preservation
                target_faces = get_many_faces(frame) if many_faces else [get_one_face(frame)]
                
                # Process face swap
                result_frame = process_frame(source_face, frame)
                
                # Apply mouth mask if landmarks available
                for target_face in target_faces:
                    if target_face and hasattr(target_face, 'landmark_2d_106'):
                        mouth_mask_img = create_mouth_mask(target_face.landmark_2d_106, frame.shape)
                        if mouth_mask_img is not None:
                            # Blend original mouth back
                            mask_3d = cv2.merge([mouth_mask_img, mouth_mask_img, mouth_mask_img]) / 255.0
                            result_frame = result_frame * (1 - mask_3d) + frame * mask_3d
                            result_frame = result_frame.astype(np.uint8)
            else:
                result_frame = process_frame(source_face, frame)
            
            cv2.imwrite(frame_path, result_frame)
            
            if i % 30 == 0:
                print(f"Processed {i+1}/{len(temp_frame_paths)} frames")
        
        create_video(target_path)
        restore_audio(target_path, output_path)
        clean_temp(target_path)
        
    else:
        print("Processing image...")
        target_img = cv2.imread(target_path)
        
        if mouth_mask:
            target_face = get_one_face(target_img)
            result_frame = process_frame(source_face, target_img)
            
            # Apply mouth mask
            if target_face and hasattr(target_face, 'landmark_2d_106'):
                mouth_mask_img = create_mouth_mask(target_face.landmark_2d_106, target_img.shape)
                if mouth_mask_img is not None:
                    mask_3d = cv2.merge([mouth_mask_img, mouth_mask_img, mouth_mask_img]) / 255.0
                    result_frame = result_frame * (1 - mask_3d) + target_img * mask_3d
                    result_frame = result_frame.astype(np.uint8)
        else:
            result_frame = process_frame(source_face, target_img)
        
        cv2.imwrite(output_path, result_frame)
    
    print(f"Face swap completed: {output_path}")

print("Fixed face swap with working mouth mask ready!")

In [ ]:
# Usage examples
source_image = "/content/source.jpg"
target_media = "/content/target.mp4"
output_file = "/content/result.mp4"

# Face swap with mouth mask (preserves original mouth)
face_swap_with_mouth_mask(source_image, target_media, output_file, mouth_mask=True)

# Download result
from google.colab import files
files.download(output_file)